In [1]:
import argparse
from gc import freeze

import torch
import numpy as np
import pandas as pd

from recbole.quick_start import run_recbole, load_data_and_model

In [8]:
pip freeze

absl-py==2.1.0
aiosignal==1.3.1
anyio @ file:///C:/b/abs_31grv7a7bl/croot/anyio_1729121290992/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci/argon2-cffi-bindings_1644551690056/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
astunparse==1.6.3
async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/async-lru_1699554572212/work
attrs @ file:///C:/b/abs_24xj44kobp/croot/attrs_1729089434205/work
Babel @ file:///C:/b/abs_a2shv_3tqi/croot/babel_1671782804377/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///C:/b/abs_d5wytg_p0w/croot/beautifulsoup4-split_1718029833749/work
black==24.10.0
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
Brotli @ file:///C:/b/abs_3d36mno480/croot/brotli-split_1714483178642/work
certifi @ file:///C:/b/abs_1fw_exq1si/croot/certifi_1725551736618/work/certifi
cffi @ file:///C:/b/abs_90yq4lmu83/croot/cffi_1726856448345/work
c

### Train & Valid

In [2]:
run_recbole(config_file_list=['recbole_config.yaml'])

25 Nov 18:05    INFO  ['C:\\Users\\baeku\\anaconda3\\envs\\level2-recsys-movierecommendation-recsys-06-lv3\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\baeku\\AppData\\Roaming\\jupyter\\runtime\\kernel-e60f8e51-ee63-445c-90cb-b40c1190a7ff.json']
25 Nov 18:05    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = C:/Users/baeku/Documents/GitHub/level2-recsys-movierecommendation-recsys-06-lv3/data/train\train_ratings
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.005
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyp

Train     0:   0%|                                                         | 0/2028 [00:00<?, ?it/s…

25 Nov 18:43    INFO  epoch 0 training [time: 2218.92s, train loss: 516.1648]


Evaluate   :   0%|                                                        | 0/31360 [00:00<?, ?it/s…

25 Nov 18:43    INFO  epoch 0 evaluating [time: 8.90s, valid_score: 0.325100]
25 Nov 18:43    INFO  valid result: 
recall@10 : 0.0993    ndcg@10 : 0.1542    mrr@10 : 0.3251
25 Nov 18:43    INFO  Saving current: saved\LightGCN-Nov-25-2024_18-06-07.pth


Train     1:   0%|                                                         | 0/2028 [00:00<?, ?it/s…

KeyboardInterrupt: 

### Inference

In [4]:
import torch

# CUDA 사용 가능 여부 확인
cuda_available = torch.cuda.is_available()

if cuda_available:
    print("CUDA is available. You can use GPU.")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is not available. Running on CPU.


In [2]:
# model, dataset 불러오기
model_path = 'saved/BERT4Rec-Nov-25-2024_09-05-06.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_path)
    
# device 설정
device = config.final_config_dict['device']

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [8]:
# user, item id -> token 변환 array
user_id2token = dataset.field2id_token['user_id']
item_id2token = dataset.field2id_token['item_id']
    
# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr')

In [9]:
# user id, predict item id 저장 변수
pred_list = None
user_list = None
    
model.eval()
for data in test_data:
    interaction = data[0].to(device)
    score = model.full_sort_predict(interaction)
        
    rating_pred = score.cpu().data.numpy().copy()
    user_id = interaction['user_id'].cpu().numpy()
        
    # 사용자가 상호작용한 아이템 인덱스를 가져옵니다.
    interacted_indices = matrix[user_id].indices

    # 상호작용한 아이템의 점수를 0으로 설정합니다.
    rating_pred[interacted_indices] = 0

    # 상위 10개 아이템 인덱스 추출
    ind = np.argpartition(rating_pred, -10)[-10:]
    arr_ind = rating_pred[ind]
       
    # 추출된 값들을 내림차순으로 정렬하기 위한 인덱스를 얻음
    arr_ind_argsort = np.argsort(arr_ind)[::-1]

    # 실제 값들을 정렬된 순서대로 인덱스 배열에 적용
    batch_pred_list = ind[arr_ind_argsort]
        
    # 예측값 저장
    if pred_list is None:
        pred_list = batch_pred_list
        # batch_pred_list 길이만큼 user_id를 반복
        user_list = np.repeat(user_id, len(batch_pred_list))
    else:
        pred_list = np.append(pred_list, batch_pred_list, axis=0)
        # batch_pred_list 길이만큼 user_id를 반복하여 추가
        user_list = np.append(user_list, np.repeat(user_id, len(batch_pred_list)), axis=0)


In [12]:
# 결과를 저장할 빈 리스트 초기화
final_result = []

# user_list와 pred_list에 있는 인덱스를 실제 'resume_seq'와 'recruitment_seq'로 변환
for user, item in zip(user_list, pred_list):
    # user_id2token을 사용하여 변환된 사용자 ID를 얻고
    # index_to_resume_seq을 사용하여 원래의 'resume_seq'로 변환
    original_user_seq = user_id2token[user]

    # item_id2token을 사용하여 변환된 아이템 ID를 얻고
    # index_to_recruitment_seq을 사용하여 원래의 'recruitment_seq'로 변환
    original_item_seq = item_id2token[item]

    # 최종 결과에 추가
    final_result.append((original_user_seq, original_item_seq))


In [20]:
# 결과를 DataFrame으로 변환하고 CSV 파일로 저장
final_dataframe = pd.DataFrame(final_result, columns=['user', 'item'])
final_dataframe.to_csv('../data/eval/recbole_multivae.csv', index=False)
print('Final mapping done and saved to CSV!')

Final mapping done and saved to CSV!
